In [11]:
from IPython.core.display import display, HTML

# Ajustar el ancho del notebook para mejor visualización
display(HTML("<style>.container { width:100% !important; }</style>"))



# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer



- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development.

In [19]:
import pandas as pd

dataset_path = "/content/kg_train.csv"  # Ruta en Google Colab

try:
    # Intentar cargar el dataset con diferentes separadores
    data = pd.read_csv(dataset_path, encoding="latin-1", on_bad_lines="skip")
    print("✅ Dataset cargado con éxito.")
except Exception as e:
    print(f"❌ Error al cargar el dataset: {e}")

# Mostrar información del dataset
print("\nEstructura del dataset:")
print(data.info())

# Mostrar las primeras filas para verificar
data.head()
# Reducir el tamaño del conjunto de datos para acelerar el desarrollo
data = data.head(1000)  # Tomar solo las primeras 1000 filas
print(f"✅ Nuevo tamaño del dataset: {data.shape}")

# Rellenar valores nulos si existen
data.fillna("", inplace=True)



✅ Dataset cargado con éxito.

Estructura del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7192 entries, 0 to 7191
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7192 non-null   object
 1   label   7192 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.5+ KB
None
✅ Nuevo tamaño del dataset: (1000, 2)


### Let's divide the training and test set into two partitions

In [22]:
from sklearn.model_selection import train_test_split

# Usar los nombres correctos de las columnas
X = data["text"]   # Ahora usamos "text" en lugar de "message"
y = data["label"]  # La columna de etiquetas sigue siendo "label"

# Dividir en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar la cantidad de datos en cada conjunto
print(f"✅ Conjunto de entrenamiento: {len(X_train)} muestras")
print(f"✅ Conjunto de prueba: {len(X_test)} muestras")



✅ Conjunto de entrenamiento: 800 muestras
✅ Conjunto de prueba: 200 muestras


## Data Preprocessing

In [24]:
import nltk

# Descargar las stopwords de NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

# Obtener lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

print("Ejemplo de stopwords:", list(stop_words)[:10])


Ejemplo de stopwords: ['own', 'why', 'haven', 'because', 'through', "wouldn't", 'this', 'who', 'can', 'as']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [25]:
import re
import string

def clean_text(text):
    # Eliminar etiquetas HTML
    text = re.sub(r'<.*?>', '', text)

    # Eliminar código JavaScript/CSS en comentarios
    text = re.sub(r'\/\*.*?\*\/', '', text, flags=re.DOTALL)  # Comentarios en bloque
    text = re.sub(r'\/\/.*?\n', '', text)  # Comentarios en línea

    return text

# Aplicar la función a los datos de entrenamiento y prueba
X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)

print("✅ Ejemplo de texto después de limpieza de HTML:", X_train.iloc[0])


✅ Ejemplo de texto después de limpieza de HTML: ----------- REGARDS, MR NELSON SMITH.KINDLY REPLY ME ON MY PRIVATE EMAIL ADDRESS;nelsonsmith2000@yahoo.com


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [26]:
def remove_special_chars(text):
    # Eliminar caracteres especiales y puntuaciones
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Eliminar números
    text = re.sub(r'\d+', '', text)

    # Eliminar caracteres únicos al inicio de palabras
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # Remover prefijo "b'" que a veces aparece en datos con encoding erróneo
    text = re.sub(r"^b'", '', text)

    # Convertir a minúsculas
    text = text.lower()

    # Eliminar espacios extra
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Aplicar la función
X_train = X_train.apply(remove_special_chars)
X_test = X_test.apply(remove_special_chars)

print("✅ Ejemplo después de remover caracteres especiales:", X_train.iloc[0])


✅ Ejemplo después de remover caracteres especiales: regards mr nelson smithkindly reply me on my private email addressnelsonsmithyahoocom


## Now let's work on removing stopwords
Remove the stopwords.

In [27]:
import nltk
from nltk.corpus import stopwords

# Descargar stopwords si es necesario
nltk.download('stopwords')

# Obtener lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

# Aplicar la eliminación de stopwords
X_train = X_train.apply(remove_stopwords)
X_test = X_test.apply(remove_stopwords)

print("✅ Ejemplo después de eliminar stopwords:", X_train.iloc[0])


✅ Ejemplo después de eliminar stopwords: regards mr nelson smithkindly reply private email addressnelsonsmithyahoocom


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [28]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# Aplicar lematización
X_train = X_train.apply(lemmatize_text)
X_test = X_test.apply(lemmatize_text)

print("✅ Ejemplo después de lematización:", X_train.iloc[0])


[nltk_data] Downloading package wordnet to /root/nltk_data...


✅ Ejemplo después de lematización: regard mr nelson smithkindly reply private email addressnelsonsmithyahoocom


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear representación de Bag of Words
vectorizer = CountVectorizer(max_features=1000)  # Tomar las 1000 palabras más frecuentes
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Obtener las palabras más frecuentes
word_freq = dict(zip(vectorizer.get_feature_names_out(), X_train_bow.toarray().sum(axis=0)))
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Mostrar las 10 palabras más comunes en el dataset
print("✅ Top 10 palabras más comunes en el dataset:")
for word, freq in sorted_words[:10]:
    print(f"{word}: {freq}")


✅ Top 10 palabras más comunes en el dataset:
money: 791
account: 669
bank: 644
fund: 555
business: 416
country: 387
one: 346
transaction: 344
million: 327
transfer: 325


## Extra features

In [35]:
import pandas as pd

# Crear DataFrames de entrenamiento y validación basados en la división previa
data_train = pd.DataFrame({"preprocessed_text": X_train, "label": y_train}).copy()
data_val = pd.DataFrame({"preprocessed_text": X_test, "label": y_test}).copy()

# Mostrar las primeras filas para verificar
print("✅ data_train y data_val creados correctamente")
data_train.head()

# Lista de símbolos monetarios
money_symbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])

# Lista de palabras sospechosas relacionadas con transacciones
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund",
                             "transfer", "transaction", "win", "deposit", "password"])

# Asegurar que la columna "preprocessed_text" es string
data_train["preprocessed_text"] = data_train["preprocessed_text"].astype(str)
data_val["preprocessed_text"] = data_val["preprocessed_text"].astype(str)

# Agregar características al conjunto de entrenamiento
data_train["money_mark"] = data_train["preprocessed_text"].str.contains(money_symbol_list, regex=True) * 1
data_train["suspicious_words"] = data_train["preprocessed_text"].str.contains(suspicious_words, regex=True) * 1
data_train["text_len"] = data_train["preprocessed_text"].apply(lambda x: len(x))

# Agregar características al conjunto de prueba
data_val["money_mark"] = data_val["preprocessed_text"].str.contains(money_symbol_list, regex=True) * 1
data_val["suspicious_words"] = data_val["preprocessed_text"].str.contains(suspicious_words, regex=True) * 1
data_val["text_len"] = data_val["preprocessed_text"].apply(lambda x: len(x))

# Mostrar las primeras filas para verificar
print("✅ Características extra añadidas con éxito:")
data_train.head()


✅ data_train y data_val creados correctamente
✅ Características extra añadidas con éxito:


,preprocessed_text,label,money_mark,suspicious_words,text_len
29,regard mr nelson smithkindly reply private ema...,1,1,0,75
535,able reach oscar supposed send pdb receive,0,1,0,42
695,huma abedin bim checking pat work jack jake re...,0,1,0,79
557,announced monday cant today,0,1,0,27
836,dear sirmadamdo accept sincere apology mail me...,1,1,1,1285


## How would work the Bag of Words with Count Vectorizer concept?

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

# Definir el vectorizador
vectorizer = CountVectorizer(max_features=1000)  # Tomar solo las 1000 palabras más frecuentes

# Transformar los datos
X_train_bow = vectorizer.fit_transform(data_train["preprocessed_text"])
X_test_bow = vectorizer.transform(data_val["preprocessed_text"])

# Mostrar dimensiones de la matriz resultante
print(f"✅ BoW - Dimensión de X_train: {X_train_bow.shape}")
print(f"✅ BoW - Dimensión de X_test: {X_test_bow.shape}")


✅ BoW - Dimensión de X_train: (800, 1000)
✅ BoW - Dimensión de X_test: (200, 1000)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Definir el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Transformar los datos
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train["preprocessed_text"])
X_test_tfidf = tfidf_vectorizer.transform(data_val["preprocessed_text"])

# Mostrar dimensiones de la matriz resultante
print(f"✅ TF-IDF - Dimensión de X_train: {X_train_tfidf.shape}")
print(f"✅ TF-IDF - Dimensión de X_test: {X_test_tfidf.shape}")


✅ TF-IDF - Dimensión de X_train: (800, 1000)
✅ TF-IDF - Dimensión de X_test: (200, 1000)


## And the Train a Classifier?

In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Definir el clasificador
clf = MultinomialNB()

# Entrenar el modelo con la representación TF-IDF
clf.fit(X_train_tfidf, data_train["label"])

# Realizar predicciones
y_pred = clf.predict(X_test_tfidf)

# Evaluar el modelo
accuracy = accuracy_score(data_val["label"], y_pred)
print(f"✅ Precisión del clasificador: {accuracy:.4f}")

# Reporte de clasificación
print(classification_report(data_val["label"], y_pred))


✅ Precisión del clasificador: 0.9550
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       115
           1       0.95      0.94      0.95        85

    accuracy                           0.95       200
   macro avg       0.95      0.95      0.95       200
weighted avg       0.95      0.95      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [39]:
# Entrenar y evaluar con Bag of Words
clf_bow = MultinomialNB()
clf_bow.fit(X_train_bow, data_train["label"])
y_pred_bow = clf_bow.predict(X_test_bow)
accuracy_bow = accuracy_score(data_val["label"], y_pred_bow)

# Entrenar y evaluar con TF-IDF
clf_tfidf = MultinomialNB()
clf_tfidf.fit(X_train_tfidf, data_train["label"])
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(data_val["label"], y_pred_tfidf)

print(f"📌 Precisión con Bag of Words: {accuracy_bow:.4f}")
print(f"📌 Precisión con TF-IDF: {accuracy_tfidf:.4f}")


📌 Precisión con Bag of Words: 0.9350
📌 Precisión con TF-IDF: 0.9550
